The N-sample moving average beginning with index $k$ is defined by

$$
\bar{x}_k = \frac{1}{N}\sum_{i=k}^{k+N}x_i
$$
It's obviously that 
$$
\bar{x}_{k+1} = \bar{x}_k + \frac{1}{N}\cdot(x_{k+N+1} - x_k)
$$

That means that we can calculate $x_{k+1}$ easily and efficiently from $x_k$.

But what about the standard deviation?

$$
\sigma_k = \sqrt{\frac{1}{N}}\sqrt{\sum_{i=k}^{k+N}(x_i - \bar{x}_k)^2}
$$

We can write the $(k+1)^\text{th}$ standard deviation as 

$$
\sigma_{k+1} = \sqrt{\frac{1}{N}}\sqrt{\sum_{i=k+1}^{k+N+1}(x_i - (\bar{x}_k+\delta_k))^2}
$$

where

$$
\delta_k = \frac{1}{N}\cdot(x_{k+N+1} - x_k)
$$

But what about the standard deviation?

$$
\sigma_{k+1} = \sqrt{\frac{1}{N}}\sqrt{\sum_{i=k+1}^{k+N+1}(x_i - (\bar{x}_k+\delta_k))^2}
$$



Let's look at the inner sum:

$$
\sum_{i=k+1}^{k+N+1}(x_i - (\bar{x}_k+\delta_k))^2 = 
$$

$$
\sum_{i=k}^{k+N}(x_i - (\bar{x}_k+\delta_k))^2 +
(x_{k+N+1}-(\bar{x}_k+\delta_k))^2 - (x_k-(\bar{x}_k+\delta_k))^2
$$

The leading sum of which is easily approximated:

$$
\sum_{i=k}^{k+N}(x_i - (\bar{x}_k+\delta_k))^2 = 
$$

$$
\sum_{i=k}^{k+N}( (x_i-\bar{x}_k)^2 -2 \delta_k (x_i-\bar{x}_k) + \delta_k^2 ) =
$$

$$
\sum_{i=k}^{k+N}(x_i-\bar{x}_k)^2 -
2 \delta_k \sum_{i=k}^{k+N}(x_i-\bar{x}_k) +
\sum_{i=k}^{k+N}\delta_k^2 =
$$

$$
\sum_{i=k}^{k+N}(x_i-\bar{x}_k)^2 - 0 + \mathcal{O}(\delta_k^2) \approx N\sigma_k^2
$$


Because obviously in the second term - the sum of the differences from the mean - is zero.
We can interpret this as: The standard deviation is insensitive against a small change in the mean.


---
#### Intermezzo: Verifying mean insensivity
Changes in the *disturbed standard deviation*, compared to changes in the mean.

In [52]:
series = np.array([1, 2, 3, 2, 1, 1, 5, 3, 2, 2, 1, 0, 2, 3, 5, 3, 1, 0, 2, 6, 4, 2, 5, 0])
n, std, mean = len(series), np.std(series), np.mean(series)
n, std, mean

(24, 1.6499158227686108, 2.3333333333333335)

In [64]:
[(delta/mean, np.sqrt(np.sum((series-mean+delta)**2)/n)/std-1) for delta in [0, .1, .2, .3]]

[(0.0, 0.0),
 (0.04285714285714286, 0.001835050987813558),
 (0.08571428571428572, 0.00732014650309698),
 (0.12857142857142856, 0.016396194645471818)]

---

That leaves us with the second and third term calling it $\zeta_k$ (say: "zeta kay")

$$
\zeta_k = (x_{k+N+1}-(\bar{x}_k+\delta_k))^2 - (x_k-(\bar{x}_k+\delta_k))^2 = 
$$

$$
x_{k+N+1}^2 - x_k^2 + 2(\bar{x}_k+\delta_k)(x_k-x_{k+N+1})
$$

$$
x_{k+N+1}^2 - x_k^2 - 2N(\bar{x}_k \delta_k + \delta_k^2)
$$


With that result we can write the inner sum of $\sigma_{k+1}$ as

$$
\sum_{i=k+1}^{k+N+1}(x_i - (\bar{x}_k+\delta_k))^2 \approx N\sigma_k^2 + \zeta_k
$$



Now, the $(k+1)^\text{th}$ moving standard deviation becomes:

$$
\sigma_{k+1} \approx \sqrt{\frac{1}{N}}\sqrt{N\sigma_k^2 + \zeta_k} = \sqrt{\sigma_k^2 + \frac{\zeta_k}{N}}
$$



And using only the first term of the Taylor series in $\zeta_k$ for the square root:

$$
\sigma_{k+1} \approx \sigma_k + \frac{\zeta_k}{2N\sigma_k}
$$

Time to verify the result!

In [169]:
from collections import deque
import numpy as np

N=30
q = deque([np.random.randint(6) for i in range(N)], maxlen=N)
m_k, s_k = np.mean(q), np.std(q)

In [174]:
x_kN1 = np.random.randint(6)
x_k = q[0]
x_k, x_kN1

(4, 2)

In [193]:
delta = (x_kN1-x_k)/N
zeta_k = x_kN1**2 - x_k**2 - 2 * N * delta * (m_k + delta)
s_k1 = s_k + zeta_k / ( 2 * N * sk )
m_k1 = m_k + delta
m_k1, s_k1

(1.9666666666666666, 1.3120963682898343)

In [194]:
q.append(x_kN1)
m_k, s_k = np.mean(q), np.std(q)
print("correct: m_k1=%s, s_k1=%s" % (m_k, s_k))


correct: m_k1=1.9333333333333333, s_k1=1.2364824660660938
